# Fotocasa Web Scrapping
Here i will show you how to extract apartment data from the website Fotocasa.
The idea for getting the data its pretty simple, since Fotocasa its a dynamic website we need to interact with it in order to be able to retrive the information we want, that means we need to scroll down the page and get the html source code as the scroll down action its performd.Once we have the hmtl file its a matter of using the BeautifulSoup library to find the information within the bins and storing all that in a df.When we reach the end of the page the code will click the next page buttom and move foward , repeting the process once again.

How are we going to interact with the web page? With the selenium web driver.

Obviosly every web page has its owns particularitys so in this case we the need to account for the different setups that Fotocasa has thorght the web page.

So the firts step as usal its to import the librarys we will be using.

In [367]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import lxml
import pandas as pd
import re
import numpy as np

### Importing and preparing the webdriver 
In this step its required to download the chrome web driver and indicate the path where the driver is.Finally we setup our webdriver with selenium.If you dont have it you can download it in here https://googlechromelabs.github.io/chrome-for-testing/

In [370]:
options = Options()
options.add_argument('--disable-blink-features=AutomationControlled')
chrome_driver_path = "/Users/aa/Desktop/Proyectos/Rent_Price/chromedriver"# Your path
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service)

The next chunk of code its going to be perfect example of what selenium driver can do. First we indicate the web page we want to go to. Then we use the XPATH for the cookies button to locate that in a object to then click that path.Then we go into the rent aparments and finally in the search bar we introduce Madrid as the city want to look for aparments.

In [372]:
driver.get('https://www.fotocasa.com/')# Select the web page ,in this case fotocasa.com
driver.implicitly_wait(5)  
#Click cookies button
cookie_button = driver.find_element(By.XPATH, '//button[@aria-label="Aceptar y cerrar: Aceptar nuestro procesamiento de datos y cerrar"]')
cookie_button.click()
#Select Rent apartments
alquiler= driver.find_element(By.XPATH,'//div[@class="re-HomeSearchSelector-item re-HomeSearchSelector-item--rent"]')
alquiler.click()
#Search for Madrid
input_field = driver.find_element(By.XPATH, '//input[@class="sui-AtomInput-input sui-AtomInput-input-size-m"]')
input_field.send_keys("Madrid")
time.sleep(1)
input_field.send_keys(Keys.ENTER)

element = WebDriverWait(driver=driver, timeout=5).until(
    EC.presence_of_element_located((By.XPATH, '//span[@class="re-CardPrice"]'))
)

### Function to extract the data
Once we have done that its necessary to write the function that will be extracting the features from the aparments. In this case its call extract_data_all.

In [374]:
def extract_data_combined(soup):
    import pandas as pd
    import re

    prices = []
    square_m = []
    rooms = []
    bathrooms = []
    zonas = []

    def extract_from_icons(soup):
        """Extracts data from containers with icons."""
        square_meters = []
        rooms = []
        bathrooms = []

        # Square meters
        for sqm_container in soup.find_all(
            'span', class_='re-CardFeaturesWithIcons-feature-icon re-CardFeaturesWithIcons-feature-icon--surface'
        ):
            sqm_text = sqm_container.get_text(strip=True)
            numbers = re.findall(r'\d+(?:[\.,]\d+)?', sqm_text)
            square_meters.append(''.join(numbers) if numbers else None)

        # Number of rooms
        for room_container in soup.find_all(
            'span', class_='re-CardFeaturesWithIcons-feature-icon re-CardFeaturesWithIcons-feature-icon--rooms'
        ):
            room_text = room_container.get_text(strip=True)
            numbers = re.findall(r'\d+', room_text)
            rooms.append(''.join(numbers) if numbers else None)

        # Number of bathrooms
        for bathroom_container in soup.find_all(
            'span', class_='re-CardFeaturesWithIcons-feature-icon re-CardFeaturesWithIcons-feature-icon--bathrooms'
        ):
            bathroom_text = bathroom_container.get_text(strip=True)
            numbers = re.findall(r'\d+', bathroom_text)
            bathrooms.append(''.join(numbers) if numbers else None)

        return square_meters, rooms, bathrooms

    def extract_without_icons(soup):
        """Extracts data from containers without icons."""
        square_meters = []
        rooms = []
        bathrooms = []

        feature_lists = soup.find_all('ul', class_='re-CardFeatures-wrapper')
        for feature_list in feature_lists:
            features = feature_list.find_all(
                'li', class_='re-CardFeatures-item re-CardFeatures-feature'
            )

            # Extract features in sequence
            number_of_rooms = (
                features[0].get_text(strip=True) if len(features) > 0 else None
            )
            number_of_bathrooms = (
                features[1].get_text(strip=True) if len(features) > 1 else None
            )
            square_meters_text = (
                features[2].get_text(strip=True) if len(features) > 2 else None
            )

            # Extract numeric values
            rooms.append(
                ''.join(re.findall(r'\d+', number_of_rooms)) if number_of_rooms else None
            )
            bathrooms.append(
                ''.join(re.findall(r'\d+', number_of_bathrooms)) if number_of_bathrooms else None
            )
            square_meters.append(
                ''.join(re.findall(r'\d+', square_meters_text)) if square_meters_text else None
            )

        return square_meters, rooms, bathrooms

    # Extract prices
    for price_container in soup.find_all('span', class_='re-CardPrice'):
        price_text = price_container.get_text(strip=True)
        numbers = re.findall(r'\d+(?:[\.,]\d+)?', price_text)
        prices.append(''.join(numbers) if numbers else None)

    square_m, rooms, bathrooms = extract_from_icons(soup)

    if not any(rooms) or not any(bathrooms):
        square_m, rooms, bathrooms = extract_without_icons(soup)

    # Extract neighborhood
    for i in soup.find_all('span', class_='re-I18nPropertyTitle'):
        word = i.get_text(strip=True)
        if ',' in word:
            parts = word.split(',')
            zonas.append(parts[-1].strip())
        else:
            split_word = word.split()
            zonas.append(split_word[-1])
    #Padding list
    max_length = max(len(prices), len(square_m), len(rooms), len(bathrooms), len(zonas))

    prices.extend([None] * (max_length - len(prices)))
    square_m.extend([None] * (max_length - len(square_m)))
    rooms.extend([None] * (max_length - len(rooms)))
    bathrooms.extend([None] * (max_length - len(bathrooms)))
    zonas.extend([None] * (max_length - len(zonas)))

    df = pd.DataFrame({
        'Price': prices,
        'Square Meters': square_m,
        'Rooms': rooms,
        'Bathrooms': bathrooms,
        'Zona': zonas
    })

    return df


### Code to scroll down and move to the next page while scrapping
The next part its going to scroll down the web page and use the previuous function to extract the prices
Now we are going to extract the prices for the first 30 pages.

In [376]:
all_prices = []
scroll_pause_time = 0.5 
df = pd.DataFrame(columns=['Price', 'Square Meters', 'Rooms', 'Bathrooms','Zona'])

def close_ad_iframe():
    try:
        iframes = driver.find_elements(By.TAG_NAME, 'iframe')
        for iframe in iframes:
            driver.switch_to.frame(iframe)
            driver.switch_to.default_content()
    except Exception as e:
        print(f"Error handling ad iframe: {e}")

def click_next_button():
    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '(//li[@class="sui-MoleculePagination-item"])[last()]'))
        )
        actions = ActionChains(driver)
        actions.move_to_element(next_button).click().perform()
        return True
    except Exception as e:
        print(f"Error clicking 'Next' button: {e}")
        return False
#Select the number of pages you want to scrape
a=soup.find_all('li',class_='sui-MoleculePagination-item')
number_of_pages = int(a[-2].get_text()) 
for page_number in range(number_of_pages): 
    try:
        
        last_height = driver.execute_script("return document.body.scrollHeight")
        for i in range(0, last_height, 500): 
            driver.execute_script("window.scrollTo(0, {});".format(i))
            time.sleep(scroll_pause_time)

        html_txt = driver.page_source
        soup = BeautifulSoup(html_txt, 'html.parser')
        
        pp = extract_data_combined(soup)
        df = pd.concat([df, pp], ignore_index=True)

        print(f"Page {page_number + 1} prices: {len(pp)}")
        
        close_ad_iframe()
        
        if not click_next_button():
            print("Failed to click 'Next' button. Stopping.")
            break
        
        time.sleep(3)
        
    except Exception as e:
        print(f"Error while processing page {page_number + 1}: {e}")
        break

print(f"Total prices extracted: {len(all_prices)}")


Page 1 prices: 31
Page 2 prices: 30
Page 3 prices: 30
Page 4 prices: 30
Page 5 prices: 30
Page 6 prices: 30
Page 7 prices: 30
Page 8 prices: 30
Page 9 prices: 30
Page 10 prices: 30
Page 11 prices: 30
Page 12 prices: 30
Page 13 prices: 30
Page 14 prices: 30
Page 15 prices: 30
Page 16 prices: 30
Page 17 prices: 30
Page 18 prices: 30
Page 19 prices: 30
Page 20 prices: 30
Page 21 prices: 30
Page 22 prices: 30
Page 23 prices: 30
Page 24 prices: 30
Page 25 prices: 30
Page 26 prices: 30
Page 27 prices: 30
Page 28 prices: 30
Page 29 prices: 30
Page 30 prices: 30
Error while processing page 31: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=131.0.6778.86)
Stacktrace:
0   chromedriver                        0x000000010bf4ce82 chromedriver + 6696578
1   chromedriver                        0x000000010bf44c9a chromedriver + 6663322
2   chromedriver                        0x000000010b94ce3e ch

WARNING:for some reason i dont fully understand yet, you need to open the window where the scrapping its been done and wait till it gets to end of the fist page, otherwise i have experienced mistakes in the code so keep this in mind.

In [381]:
df

,Price,Square Meters,Rooms,Bathrooms,Zona
0,4.300,208,3,2,Hispanoamérica - Bernabéu
1,985,63,3,1,Cristóbal
2,790,28,1,1,Huertas
3,1.395,45,2,1,Butarque
4,1.595,90,1,2,El Cañaveral
...,...,...,...,...,...
896,2.500,76,2,1,Castellana
897,2.250,42,1,1,Cuatro Caminos - Azca
898,2.250,51,1,1,Ríos Rosas - Nuevos Ministerios
899,2.500,60,1,1,Recoletos


In [385]:
df.to_csv('madrid_data.csv', index=False) 
